In [1]:
#===================================================================
# Imports y espacio de búsqueda
#===================================================================

import numpy as np
import json
from ultralytics import YOLO

# Definir espacio de hiperparámetros
param_space = {
    "lr0": (1e-4, 1e-2),
    "momentum": (0.85, 0.98),
    "weight_decay": (1e-6, 1e-3),
    "hsv_s": (0.0, 0.7),
    "scale": (0.0, 1.0),
    "translate": (0.0, 0.2)
}
param_keys = list(param_space.keys())
dim = len(param_keys)

def sample_particle():
    return np.array([np.random.uniform(low, high) for low, high in param_space.values()])

In [2]:
#===================================================================
# Baseline (con Hiperparámetros por defecto)
#===================================================================

def run_baseline_training(epochs=5, imgsz=640):
    """
    Entrena un modelo YOLOv8 con hiperparámetros por defecto
    para establecer una línea de base (baseline) de rendimiento.

    Retorna:
        float: El score mAP50-95 del modelo baseline.
    """
    print("--- Iniciando Entrenamiento Baseline ---")
    print(f"Entrenando con {epochs} épocas y tamaño de imagen {imgsz}...")
    
    model = YOLO("yolov8n.pt")

    # Entrena con los valores por defecto, guardando en una carpeta específica
    model.train(
        data="data.yaml",
        epochs=epochs,
        imgsz=imgsz,
        verbose=True, # Mantenlo en True para ver el progreso claramente
        name='baseline_training' # Nombra la carpeta de resultados para claridad
    ) 

    # Valida y obtén las métricas
    metrics = model.val()
    baseline_map = metrics.box.map

    print("\n--- Entrenamiento Baseline Finalizado ---")
    print(f"Baseline mAP50-95: {baseline_map:.4f}")
    
    return baseline_map

In [3]:
#====================================================================
# Función de evaluación (fitness)
#====================================================================

def evaluate_particle(position, epochs=5, imgsz=640):
    """Entrena YOLO con hiperparámetros de una partícula y devuelve fitness."""
    cfg = {k: float(v) for k, v in zip(param_keys, position)}

    overrides = dict(
        data="data.yaml",
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        optimizer="AdamW",
        lr0=cfg["lr0"],
        momentum=cfg["momentum"],
        weight_decay=cfg["weight_decay"],
        hsv_s=cfg["hsv_s"],
        scale=cfg["scale"],
        translate=cfg["translate"],
        #device=0,
        verbose=False
    )

    model = YOLO("yolov8n.pt")
    model.train(**overrides)
    metrics = model.val()

    fitness = metrics.box.map  # mAP50-95
    return fitness, cfg, metrics


In [4]:
#===================================================================
# Metaheurísticas
#===================================================================

# PSO
def iterarPSO(maxIter, iter, dim, population, best, pBest, vel, ub0):
    Vmax = ub0 * 0.1
    wMax, wMin = 0.9, 0.1
    c1, c2 = 2, 2

    w = wMax - iter * ((wMax - wMin) / maxIter)
    r1 = np.random.rand(population.shape[0], dim)
    r2 = np.random.rand(population.shape[0], dim)

    vel = (
        w * vel
        + c1 * r1 * (pBest - population)
        + c2 * r2 * (best - population)
    )
    vel = np.clip(vel, -Vmax, Vmax)
    population = population + vel
    return population, vel

# WOA
import math, random
def iterarWOA(maxIter, iter, dim, population, best):
    a = 2 - (2 * iter / maxIter)
    b = 1
    new_population = []

    for individual in population:
        p = random.uniform(0, 1)
        r = random.uniform(0, 1)
        l = random.uniform(-1, 1)
        A = 2 * a * r - a
        C = 2 * random.uniform(0, 1)

        if p < 0.5:
            if abs(A) < 1:  # encircle best
                D = [abs(C * best[j] - individual[j]) for j in range(dim)]
                new_individual = [best[j] - A * D[j] for j in range(dim)]
            else:  # random search
                rand_idx = random.randint(0, len(population) - 1)
                rand_ind = population[rand_idx]
                D = [abs(C * rand_ind[j] - individual[j]) for j in range(dim)]
                new_individual = [rand_ind[j] - A * D[j] for j in range(dim)]
        else:  # spiral update
            D_prime = [best[j] - individual[j] for j in range(dim)]
            spiral_component = math.exp(b * l) * math.cos(2 * math.pi * l)
            new_individual = [D_prime[j] * spiral_component + best[j] for j in range(dim)]

        new_population.append(new_individual)
    return np.array(new_population)

# GWO
def iterarGWO(maxIter, iter, dim, population, fitness):
    population = np.array(population)
    fitness = np.array(fitness)

    # Parámetro 'a' decrece linealmente de 2 a 0
    a = 2 - iter * (2 / maxIter)

    # Ordenar posiciones según fitness (descendente para maximizar)
    sorted_indices = np.argsort(fitness)[::-1]

    # Alpha, Beta, Delta wolves
    Xalfa = population[sorted_indices[0]]
    Xbeta = population[sorted_indices[1]]
    Xdelta = population[sorted_indices[2]]

    # Random values para todos los cálculos
    r1 = np.random.uniform(0.0, 1.0, (population.shape[0], dim, 3))
    r2 = np.random.uniform(0.0, 1.0, (population.shape[0], dim, 3))

    # Calcular A y C
    A = 2 * a * r1 - a
    C = 2 * r2

    # Distancias a los 3 lobos líderes
    d_alfa = np.abs(C[:, :, 0] * Xalfa - population)
    d_beta = np.abs(C[:, :, 1] * Xbeta - population)
    d_delta = np.abs(C[:, :, 2] * Xdelta - population)

    # Actualización de posiciones
    X1 = Xalfa - A[:, :, 0] * d_alfa
    X2 = Xbeta - A[:, :, 1] * d_beta
    X3 = Xdelta - A[:, :, 2] * d_delta

    # Promedio de las 3 guías
    population = (X1 + X2 + X3) / 3

    return population

#FA
def iterarFA(maxIter, iter, dim, population, fitness, alpha=0.5, beta0=1.0, gamma=1.0):
    """
    Implementación simple del Firefly Algorithm para tu marco de optimización.
    - population: matriz (n_individuos, dim)
    - fitness: lista o array con el valor de cada individuo
    - alpha: factor de aleatoriedad
    - beta0: atractividad base
    - gamma: coeficiente de absorción de luz
    """
    n_fireflies = population.shape[0]
    new_population = np.copy(population)
    fitness = np.array(fitness)

    # Ordenar luciérnagas por brillo (fitness)
    sorted_indices = np.argsort(fitness)[::-1]  # descendente (mayor fitness = más brillo)
    population = population[sorted_indices]
    fitness = fitness[sorted_indices]

    # Actualizar cada luciérnaga según las más brillantes
    for i in range(n_fireflies):
        for j in range(n_fireflies):
            if fitness[j] > fitness[i]:  # luciérnaga j más brillante
                r = np.linalg.norm(population[i] - population[j])
                beta = beta0 * np.exp(-gamma * (r ** 2))
                e = np.random.randn(dim)
                # Movimiento
                new_population[i] += beta * (population[j] - population[i]) + alpha * e

    # Retornar nueva población
    return new_population

In [5]:
#===================================================================
# Loop general de optimización
#===================================================================

def run_metaheuristic(name="PSO", n_particles=4, max_iter=3, epochs=5):
    population = np.array([sample_particle() for _ in range(n_particles)])
    vel = np.zeros_like(population)
    pBest, pBest_scores = np.copy(population), np.full(n_particles, -np.inf)
    gBest, gBest_score = None, -np.inf

    history = []

    for it in range(max_iter):
        print(f"\nIteración {it+1}/{max_iter}")
        for i, particle in enumerate(population):
            fitness, cfg, metrics = evaluate_particle(particle, epochs=epochs)

            if fitness > pBest_scores[i]:
                pBest[i] = particle
                pBest_scores[i] = fitness
            if fitness > gBest_score:
                gBest, gBest_score = particle, fitness

            history.append({"iter": it, "particle": i, "fitness": fitness, "cfg": cfg})

        # Elegir algoritmo
        if name == "PSO":
            ub0 = np.array([high for _, high in param_space.values()])
            population, vel = iterarPSO(max_iter, it, dim, population, gBest, pBest, vel, ub0)
        elif name == "WOA":
            population = iterarWOA(max_iter, it, dim, population, gBest)
        elif name == "GWO":
            current_fitness = [h["fitness"] for h in history if h["iter"] == it]
            population = iterarGWO(max_iter, it, dim, population, current_fitness)
        elif name == "FA":
            current_fitness = [h["fitness"] for h in history if h["iter"] == it]
            population = iterarFA(max_iter, it, dim, population, current_fitness)
            
        # Clipping a los rangos definidos
        for i in range(population.shape[0]):
            for j, (low, high) in enumerate(param_space.values()):
                population[i, j] = np.clip(population[i, j], low, high)

        print(f"  Mejor global hasta ahora: {gBest_score:.4f}")

    with open(f"{name}_results.json", "w") as f:
        json.dump(history, f, indent=4)

    return gBest, gBest_score, history


In [6]:
#===================================================================
# Ejecución Maestra del Experimento
#===================================================================

# --- CONFIGURACIÓN DEL EXPERIMENTO ---
# Define aquí todos los algoritmos que quieres probar
algorithms_to_test = ["PSO", "WOA", "GWO", "FA"]
num_particles = 3
num_iterations = 1
num_epochs = 1

# Diccionario para guardar todos los resultados
all_results = {}


# --- PASO 1: Establecer la línea de base (baseline) UNA SOLA VEZ ---
print("INICIANDO EXPERIMENTO: OBTENIENDO BASELINE\n" + "="*50)
baseline_score = run_baseline_training(epochs=num_epochs)
print(f"\nEl score a superar es: {baseline_score:.4f}\n")


# --- PASO 2: Bucle para ejecutar cada metaheurística ---
for algo_name in algorithms_to_test:
    print(f"\nINICIANDO OPTIMIZACIÓN CON '{algo_name}'\n" + "="*50)
    
    best_cfg, best_score, hist = run_metaheuristic(
        name=algo_name, 
        n_particles=num_particles, 
        max_iter=num_iterations,
        epochs=num_epochs
    )
    
    # Guarda los resultados de este algoritmo
    all_results[algo_name] = {
        'score': best_score,
        'config': dict(zip(param_keys, best_cfg)) # Guardamos como dict para más claridad
    }
    print(f"--- Optimización con '{algo_name}' finalizada. Mejor score: {best_score:.4f} ---")


# --- PASO 3: Mostrar un resumen final comparativo ---
print("\n\n" + "="*60)
print("--- RESUMEN FINAL DEL EXPERIMENTO ---")
print("="*60)

print(f"\nBaseline mAP50-95 (Default): {baseline_score:.4f}")
print("-" * 30)

# Imprimir los resultados de cada algoritmo y su mejora
for algo_name, result in all_results.items():
    score = result['score']
    improvement = ((score - baseline_score) / baseline_score) * 100
    
    print(f"\nAlgoritmo: {algo_name}")
    print(f"  - Mejor mAP50-95: {score:.4f}")
    print(f"  - Mejora sobre Baseline: {improvement:.2f}%")
    print("  - Mejor Configuración:")
    for k, v in result['config'].items():
        print(f"    {k}: {v:.6f}")

print("\n" + "="*60)
print("--- FIN DEL EXPERIMENTO ---")
print("="*60)

INICIANDO EXPERIMENTO: OBTENIENDO BASELINE
--- Iniciando Entrenamiento Baseline ---
Entrenando con 1 épocas y tamaño de imagen 640...


100%|██████████| 6.25M/6.25M [00:00<00:00, 10.8MB/s]


New https://pypi.org/project/ultralytics/8.3.209 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.161  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=baseline_training, nbs=64, nms=False, opset=None, opti

train: Scanning C:\Users\mati1\Desktop\Repositorios\hybrid-vehicle-yolo\dataset\train\labels... 2704 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2704/2704 [00:06<00:00, 447.65it/s]


train: New cache created: C:\Users\mati1\Desktop\Repositorios\hybrid-vehicle-yolo\dataset\train\labels.cache
val: Fast image access  (ping: 0.40.1 ms, read: 34.922.1 MB/s, size: 45.9 KB)


...

                   all        300       2568      0.245      0.135     0.0548     0.0251
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 8.2ms postprocess per image
Results saved to runs\detect\train13
Ultralytics 8.3.161  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
Model summary (fused): 72 layers, 3,009,743 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 308.491.0 MB/s, size: 35.2 KB)


val: Scanning C:\Users\mati1\Desktop\Repositorios\hybrid-vehicle-yolo\dataset\valid\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: C:\Users\mati1\Desktop\Repositorios\hybrid-vehicle-yolo\dataset\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: C:\Users\mati1\Desktop\Repositorios\hybrid-vehicle-yolo\dataset\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.27it/s]


                   all        300       2568      0.245      0.135     0.0548     0.0251
Speed: 0.5ms preprocess, 2.4ms inference, 0.0ms loss, 7.0ms postprocess per image
Results saved to runs\detect\train132
  Mejor global hasta ahora: 0.0452
--- Optimización con 'FA' finalizada. Mejor score: 0.0452 ---


--- RESUMEN FINAL DEL EXPERIMENTO ---

Baseline mAP50-95 (Default): 0.0758
------------------------------

Algoritmo: PSO
  - Mejor mAP50-95: 0.0290
  - Mejora sobre Baseline: -61.69%
  - Mejor Configuración:
    lr0: 0.005724
    momentum: 0.970328
    weight_decay: 0.000072
    hsv_s: 0.060991
    scale: 0.020218
    translate: 0.166524

Algoritmo: WOA
  - Mejor mAP50-95: 0.0403
  - Mejora sobre Baseline: -46.82%
  - Mejor Configuración:
    lr0: 0.003223
    momentum: 0.897282
    weight_decay: 0.000571
    hsv_s: 0.307021
    scale: 0.988374
    translate: 0.020409

Algoritmo: GWO
  - Mejor mAP50-95: 0.0290
  - Mejora sobre Baseline: -61.69%
  - Mejor Configuración:
    lr0: 0.00